In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import re

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-notifications")
driver = webdriver.Chrome(options=options)

In [3]:
IPPool = []
for i in range(1,6):
    # 用迴圈逐一打開分頁
    url = 'http://free-proxy.cz/zh/proxylist/country/US/http/ping/all/{}'.format(i)
    print('Dealing with {}'.format(url))
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    for j in soup.select('tbody > tr'):
        # 用正則表達式抓取IP
        if re.findall('[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}', str(j)):
            IP = re.findall('[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}', str(j))[0]
            Port = re.findall('class="fport" style="">(.*?)</span>', str(j))[0]
            IPPool.append(pd.DataFrame([{'IP':IP, 'Port':Port}]))
    print('There are {} IPs in Pool'.format(len(IPPool)))
IPPool = pd.concat(IPPool, ignore_index=True)
IPPool

Dealing with http://free-proxy.cz/zh/proxylist/country/US/http/ping/all/1
There are 35 IPs in Pool
Dealing with http://free-proxy.cz/zh/proxylist/country/US/http/ping/all/2
There are 70 IPs in Pool
Dealing with http://free-proxy.cz/zh/proxylist/country/US/http/ping/all/3
There are 105 IPs in Pool
Dealing with http://free-proxy.cz/zh/proxylist/country/US/http/ping/all/4
There are 140 IPs in Pool
Dealing with http://free-proxy.cz/zh/proxylist/country/US/http/ping/all/5
There are 175 IPs in Pool


,IP,Port
0,107.1.80.135,80
1,198.199.83.163,80
2,104.129.194.38,8800
3,47.89.153.213,80
4,50.206.25.106,80
...,...,...
170,162.55.106.185,3128
171,201.140.238.197,3128
172,173.46.67.172,58517
173,3.214.165.79,8090


In [4]:
ActIps = []
for IP, Port in zip(IPPool['IP'],IPPool['Port']):
    proxy = {'http':'http://'+ IP + ':' + Port,
            'https':'https://'+ IP + ':' + Port} 
    try:
        # 隨機找的一篇新聞即可
        url = 'https://www.ncbi.nlm.nih.gov/pmc/articles/pmid/33942874'
        resp = requests.get(url, proxies=proxy, timeout=2)
        if str(resp.status_code) == '200':
            ActIps.append(pd.DataFrame([{'IP':IP, 'Port':Port}]))
            print('Succed: {}:{}'.format(IP, Port))
        else:
            print('Failed: {}:{}'.format(IP, Port))
    except:
            print('Failed: {}:{}'.format(IP, Port))
ActIps = pd.concat(ActIps, ignore_index=True)
ActIps  

Failed: 107.1.80.135:80
Failed: 198.199.83.163:80
Failed: 104.129.194.38:8800
Failed: 47.89.153.213:80
Failed: 50.206.25.106:80
Failed: 50.206.25.111:80
Failed: 50.206.25.104:80
Failed: 50.206.25.107:80
Failed: 50.206.25.108:80
Failed: 50.206.25.109:80
Failed: 50.206.25.110:80
Failed: 50.206.25.105:80
Failed: 18.235.220.172:8080
Failed: 45.42.177.22:3128
Failed: 52.149.152.236:80
Failed: 45.42.177.4:3128
Failed: 45.42.177.5:3128
Failed: 18.235.108.88:8080
Failed: 98.116.152.143:3128
Failed: 67.207.83.225:80
Failed: 35.173.66.247:3128
Failed: 68.188.59.198:80
Failed: 207.246.121.161:3128
Failed: 54.196.123.146:80
Failed: 35.239.150.193:8080
Failed: 40.121.241.218:3128
Failed: 18.235.36.22:80
Failed: 209.97.150.167:8080
Failed: 34.205.43.237:38888
Failed: 47.184.181.61:3128
Failed: 52.15.239.118:38888
Failed: 173.230.139.107:8080
Failed: 67.253.148.206:8080
Failed: 20.81.62.32:3128
Failed: 18.220.250.120:38888
Failed: 3.239.121.146:38888
Failed: 34.121.55.127:8080
Failed: 74.143.245.221:

ValueError: No objects to concatenate